In [8]:
# Initial imports

import os
import json
import pandas as pd
import nltk as nltk
nltk.download('vader_lexicon')

import alpaca_trade_api as tradeapi
import seaborn as sns
import yfinance as yf

import pandas as pd
import datetime
import time

from pandas_datareader import data as pdr
from newsapi import NewsApiClient
from dotenv import load_dotenv

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import alpaca_trade_api as tradeapi

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\munir\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\munir\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
# Pull Sp500 compaies from wikipedia

Sp500_cos=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = Sp500_cos[0]
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [10]:
# Create a list of sp500 tickers 

symbols = df['Symbol'].values.tolist()
print(symbols[:15])


['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK']


In [11]:
# Pull daily volume of sp500 tickers for 6 months period

start_date = '2020-10-09'
end_date = '2021-04-09'

data = yf.download(symbols,start_date,end_date)
daily_vol = data["Volume"]


[*********************100%***********************]  505 of 505 completed

3 Failed downloads:
- FOX: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [5]:
# Daily volume of sp500 tickers

daily_vol

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-09,709000.0,65281000.0,840400.0,100506900.0,6005800.0,759900.0,150300.0,3060700.0,1312500.0,1853800.0,...,1464500.0,24073800.0,29228600.0,935900.0,722100.0,1169700.0,1032300.0,216200.0,910200.0,1620900.0
2020-10-12,1003700.0,39852500.0,720400.0,240226800.0,7314900.0,816400.0,243600.0,4201800.0,1524300.0,3149700.0,...,2274300.0,4829300.0,17912800.0,582300.0,775300.0,976900.0,964000.0,277900.0,980500.0,1582300.0
2020-10-13,915500.0,63372200.0,753400.0,262330500.0,7851700.0,703300.0,191700.0,4773600.0,1634000.0,2438100.0,...,2608900.0,2457400.0,20995100.0,600500.0,768700.0,847600.0,1211300.0,227300.0,1123200.0,1515800.0
2020-10-14,906700.0,37647700.0,581800.0,151062300.0,5864500.0,737300.0,241500.0,4191600.0,1443800.0,2032900.0,...,1722700.0,2473200.0,17756500.0,626900.0,776900.0,1317000.0,858500.0,345600.0,1348800.0,1319500.0
2020-10-15,723000.0,33776100.0,697300.0,112559200.0,6190900.0,1270600.0,264000.0,3518500.0,1273300.0,2042800.0,...,2039700.0,2532500.0,23251600.0,657300.0,532100.0,2427900.0,991000.0,204300.0,1852700.0,1363800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-05,1096300.0,33657000.0,998300.0,88651200.0,10584400.0,997300.0,233800.0,6314600.0,2631300.0,2982500.0,...,3538400.0,1885000.0,23264800.0,1687800.0,828000.0,1239200.0,788900.0,226000.0,1069500.0,1212600.0
2021-04-06,1623200.0,31099400.0,901400.0,80171300.0,8522500.0,795800.0,245700.0,5146400.0,1786600.0,1878300.0,...,2180300.0,2587300.0,18926300.0,1232600.0,752500.0,2280000.0,795800.0,181900.0,907300.0,1802500.0
2021-04-07,954600.0,27792500.0,443900.0,83466700.0,8037300.0,765800.0,148000.0,4313000.0,2736300.0,1576800.0,...,2343300.0,1782500.0,22385000.0,2048600.0,651300.0,1095600.0,691400.0,203200.0,659000.0,1660500.0


In [6]:
# Characterstics of the daily volume data to find mean volume

daily_volume = daily_vol.describe()
daily_volume

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
count,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,124.000000,1.240000e+02,1.240000e+02,1.240000e+02,...,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02
mean,1.654640e+06,6.469185e+07,9.839315e+05,1.110259e+08,7.845515e+06,1.094581e+06,305266.129032,5.006737e+06,2.010431e+06,2.680306e+06,...,2.765739e+06,3.339827e+06,3.063438e+07,1.415203e+06,9.800629e+05,1.666755e+06,1.174486e+06,3.675839e+05,1.765769e+06,1.891740e+06
std,7.205078e+05,4.107133e+07,4.016361e+05,3.506743e+07,3.267661e+06,4.666253e+05,142149.077756,1.896581e+06,7.226737e+05,8.871525e+05,...,1.183190e+06,3.566522e+06,1.008451e+07,7.851751e+05,3.638007e+05,5.558471e+05,4.827968e+05,1.797886e+05,1.222878e+06,7.722039e+05
min,6.360000e+05,2.595620e+07,3.080000e+05,4.669130e+07,1.378600e+06,2.958000e+05,65700.000000,1.451500e+06,5.281000e+05,5.892000e+05,...,5.984000e+05,9.626000e+05,8.039000e+06,2.176000e+05,2.497000e+05,3.404000e+05,2.838000e+05,1.383000e+05,3.277000e+05,4.174000e+05
25%,1.167025e+06,3.980952e+07,7.475250e+05,8.788975e+07,5.877475e+06,7.806250e+05,220200.000000,3.872650e+06,1.602325e+06,2.040325e+06,...,2.009375e+06,1.900950e+06,2.395198e+07,8.882250e+05,7.619750e+05,1.299525e+06,8.416000e+05,2.409500e+05,1.210875e+06,1.457550e+06
50%,1.520900e+06,5.246385e+07,8.812000e+05,1.029580e+08,7.243400e+06,9.883500e+05,274900.000000,4.598750e+06,1.877500e+06,2.544550e+06,...,2.500050e+06,2.592550e+06,2.914360e+07,1.238900e+06,9.303500e+05,1.570050e+06,1.068650e+06,3.128500e+05,1.539200e+06,1.673350e+06
75%,1.926275e+06,7.564198e+07,1.103100e+06,1.244393e+08,9.103875e+06,1.297450e+06,349800.000000,5.658650e+06,2.263825e+06,3.143850e+06,...,3.258500e+06,3.431575e+06,3.529735e+07,1.712150e+06,1.133450e+06,1.948975e+06,1.413150e+06,4.743750e+05,1.879075e+06,2.124350e+06
max,5.559900e+06,3.196076e+08,3.368300e+06,2.623305e+08,2.584900e+07,3.229200e+06,973900.000000,1.399070e+07,5.948000e+06,5.317000e+06,...,9.195000e+06,3.209330e+07,8.404400e+07,5.957100e+06,2.677900e+06,4.513800e+06,3.544900e+06,1.189700e+06,1.246410e+07,6.111000e+06


In [7]:
# Daily volume data data is transposed 

daily_volume = daily_volume.transpose()
daily_volume

,count,mean,std,min,25%,50%,75%,max
A,124.0,1.654640e+06,7.205078e+05,636000.0,1167025.0,1520900.0,1926275.0,5559900.0
AAL,124.0,6.469185e+07,4.107133e+07,25956200.0,39809525.0,52463850.0,75641975.0,319607600.0
AAP,124.0,9.839315e+05,4.016361e+05,308000.0,747525.0,881200.0,1103100.0,3368300.0
AAPL,124.0,1.110259e+08,3.506743e+07,46691300.0,87889750.0,102957950.0,124439325.0,262330500.0
ABBV,124.0,7.845515e+06,3.267661e+06,1378600.0,5877475.0,7243400.0,9103875.0,25849000.0
...,...,...,...,...,...,...,...,...
YUM,124.0,1.666755e+06,5.558471e+05,340400.0,1299525.0,1570050.0,1948975.0,4513800.0
ZBH,124.0,1.174486e+06,4.827968e+05,283800.0,841600.0,1068650.0,1413150.0,3544900.0
ZBRA,124.0,3.675839e+05,1.797886e+05,138300.0,240950.0,312850.0,474375.0,1189700.0
ZION,124.0,1.765769e+06,1.222878e+06,327700.0,1210875.0,1539200.0,1879075.0,12464100.0


In [67]:
# Daily volume data data is put into pandas dataframe

daily_volume = pd.DataFrame(daily_volume)
daily_volume

,count,mean,std,min,25%,50%,75%,max
A,124.0,1.654640e+06,7.205078e+05,636000.0,1167025.0,1520900.0,1926275.0,5559900.0
AAL,124.0,6.466721e+07,4.107944e+07,25956200.0,39745975.0,52394000.0,75641975.0,319607600.0
AAP,124.0,9.840532e+05,4.017711e+05,308000.0,747525.0,881200.0,1103100.0,3368300.0
AAPL,124.0,1.110259e+08,3.506743e+07,46691300.0,87889750.0,102957950.0,124439325.0,262330500.0
ABBV,124.0,7.844761e+06,3.266904e+06,1378600.0,5877325.0,7242800.0,9103875.0,25849000.0
...,...,...,...,...,...,...,...,...
YUM,124.0,1.666755e+06,5.558471e+05,340400.0,1299525.0,1570050.0,1948975.0,4513800.0
ZBH,124.0,1.174550e+06,4.827805e+05,283800.0,841600.0,1068650.0,1415175.0,3544900.0
ZBRA,124.0,3.675839e+05,1.797886e+05,138300.0,240950.0,312850.0,474375.0,1189700.0
ZION,124.0,1.765769e+06,1.222878e+06,327700.0,1210875.0,1539200.0,1879075.0,12464100.0


In [68]:
# Daily volume dataframe index is set as Symbols 

daily_volume.index = df["Symbol"]
daily_volume

,count,mean,std,min,25%,50%,75%,max
Symbol,,,,,,,,
MMM,124.0,1.654640e+06,7.205078e+05,636000.0,1167025.0,1520900.0,1926275.0,5559900.0
ABT,124.0,6.466721e+07,4.107944e+07,25956200.0,39745975.0,52394000.0,75641975.0,319607600.0
ABBV,124.0,9.840532e+05,4.017711e+05,308000.0,747525.0,881200.0,1103100.0,3368300.0
ABMD,124.0,1.110259e+08,3.506743e+07,46691300.0,87889750.0,102957950.0,124439325.0,262330500.0
ACN,124.0,7.844761e+06,3.266904e+06,1378600.0,5877325.0,7242800.0,9103875.0,25849000.0
...,...,...,...,...,...,...,...,...
YUM,124.0,1.666755e+06,5.558471e+05,340400.0,1299525.0,1570050.0,1948975.0,4513800.0
ZBRA,124.0,1.174550e+06,4.827805e+05,283800.0,841600.0,1068650.0,1415175.0,3544900.0
ZBH,124.0,3.675839e+05,1.797886e+05,138300.0,240950.0,312850.0,474375.0,1189700.0


In [69]:
# Daily volume dataframe mean column is renamed to Avg Daily Vol 

daily_volume = daily_volume.rename(columns={'mean': 'Avg Daily Vol'})
daily_volume 

,count,Avg Daily Vol,std,min,25%,50%,75%,max
Symbol,,,,,,,,
MMM,124.0,1.654640e+06,7.205078e+05,636000.0,1167025.0,1520900.0,1926275.0,5559900.0
ABT,124.0,6.466721e+07,4.107944e+07,25956200.0,39745975.0,52394000.0,75641975.0,319607600.0
ABBV,124.0,9.840532e+05,4.017711e+05,308000.0,747525.0,881200.0,1103100.0,3368300.0
ABMD,124.0,1.110259e+08,3.506743e+07,46691300.0,87889750.0,102957950.0,124439325.0,262330500.0
ACN,124.0,7.844761e+06,3.266904e+06,1378600.0,5877325.0,7242800.0,9103875.0,25849000.0
...,...,...,...,...,...,...,...,...
YUM,124.0,1.666755e+06,5.558471e+05,340400.0,1299525.0,1570050.0,1948975.0,4513800.0
ZBRA,124.0,1.174550e+06,4.827805e+05,283800.0,841600.0,1068650.0,1415175.0,3544900.0
ZBH,124.0,3.675839e+05,1.797886e+05,138300.0,240950.0,312850.0,474375.0,1189700.0


In [70]:
# Top 30 tickers by avarage daily volume in last 6 months 

top_30_stocks = daily_volume["Avg Daily Vol"].nlargest(n=30)

In [71]:
# A list of top 30 tickers by volume is created

top_30_stocks_list = top_30_stocks.index.tolist() # Convert index to list
top_30_stocks_list

['ABMD',
 'FLIR',
 'ESS',
 'ABT',
 'ADP',
 'CPB',
 'AXP',
 'SYF',
 'TSCO',
 'PYPL',
 'WM',
 'HBAN',
 'XEL',
 'MAA',
 'MCHP',
 'OMC',
 'VTR',
 'MCO',
 'CME',
 'BSX',
 'RE',
 'FBHS',
 'EVRG',
 'TRMB',
 'MPWR',
 'ADSK',
 'VNO',
 'CF',
 'ULTA',
 'JCI']

In [72]:
# Sentiment Analysis on list of top 30 tickers by volume 

# Load .env enviroment variables
load_dotenv()

# Read your api key environment variable
# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWSAPI_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [73]:
# Articles are pulled from NewsAPI by running for Loop and a dictionary name  nlp_stocks consisting of keys as tickers and values as corresponding articles is created 

# Fetch the news articles

nlp_stocks = {}
            
for i in top_30_stocks_list:
        
    keyword = i  

    all_articles =newsapi.get_everything(q=keyword,
                                        language="en",
                                        page_size=100,
                                        from_param='2021-03-13',
                                        to='2021-04-12',
                                        sort_by="relevancy")
            
    nlp_stocks[i] = all_articles
    
    
    
#save json
    with open('headline.json', 'w') as json_file:
        json.dump(nlp_stocks, json_file)        

In [74]:
# nlp_stocks dictionary is run 

nlp_stocks

{'ABMD': {'status': 'ok',
  'totalResults': 7,
  'articles': [{'source': {'id': None, 'name': 'Business Wire'},
    'author': None,
    'title': 'Abiomed Fourth Quarter Fiscal 2021 Earnings and Conference Call Notification',
    'description': 'DANVERS, Mass.--(BUSINESS WIRE)--Abiomed announces that on Thursday, April 29, 2021 the Company will release financial results for the fourth quarter of fiscal 2021.',
    'url': 'https://www.businesswire.com/news/home/20210408005111/en/Abiomed-Fourth-Quarter-Fiscal-2021-Earnings-and-Conference-Call-Notification',
    'urlToImage': 'https://mms.businesswire.com/media/20210408005111/en/722993/23/AbioLogo_TAGLINE_COLOR.jpg',
    'publishedAt': '2021-04-08T11:06:16Z',
    'content': 'DANVERS, Mass.--(BUSINESS WIRE)--Abiomed, Inc. (NASDAQ: ABMD) announced that on Thursday, April 29, 2021, the Company will release financial results for the fourth quarter of fiscal 2021. The Company… [+2426 chars]'},
   {'source': {'id': None, 'name': 'Seeking Alpha'}

In [75]:
# Length of nlp_stocks is checked to make sure all the top 30 tickers are available as keys    

nlp_stocks.keys()

dict_keys(['ABMD', 'FLIR', 'ESS', 'ABT', 'ADP', 'CPB', 'AXP', 'SYF', 'TSCO', 'PYPL', 'WM', 'HBAN', 'XEL', 'MAA', 'MCHP', 'OMC', 'VTR', 'MCO', 'CME', 'BSX', 'RE', 'FBHS', 'EVRG', 'TRMB', 'MPWR', 'ADSK', 'VNO', 'CF', 'ULTA', 'JCI'])

In [76]:
# Create top 30 tickers sentiment scores DataFrame 

stock_sentiments = []
stock_list=[]

for i in top_30_stocks_list:
 
    stock_list.append(i)
        
    for article in nlp_stocks[i]["articles"]:
        
        try:
            text = article["content"]
            date = article["publishedAt"]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
        
            sentiments = [i, date,text, compound, pos, neg, neu]
            
            stock_sentiments.append(sentiments)
        
            print(sentiments)    
        
        except AttributeError:
            pass

['ABMD', '2021-04-08T11:06:16Z', 'DANVERS, Mass.--(BUSINESS WIRE)--Abiomed, Inc. (NASDAQ: ABMD) announced that on Thursday, April 29, 2021, the Company will release financial results for the fourth quarter of fiscal 2021. The Company… [+2426 chars]', 0.0, 0.0, 0.0, 1.0]
['ABMD', '2021-03-16T15:39:23Z', '<ul><li>According to an update posted on the federal register for clinical trials, Abbott (ABT +0.2%\r\n) has suspended the recruitment for SHIELD II clinical trial citing reports of ‘decreased impelle… [+480 chars]', -0.4767, 0.0, 0.094, 0.906]
['ABMD', '2021-04-08T14:45:20Z', 'The financial regulations require hedge funds and wealthy investors that exceeded the $100 million holdings threshold to file a report that shows their positions at the end of every quarter. Even tho… [+6029 chars]', 0.3612, 0.072, 0.0, 0.928]
['ABMD', '2021-04-06T17:42:16Z', 'The Insider Monkey team has completed processing the quarterly 13F filings for the December quarter submitted by the hedge funds and othe

In [77]:
# length of stock_sentiments list is checked 

len(stock_sentiments)

1955

In [78]:
# Create the pandas DataFrame of nlp_stocks with sentiments

nlp_stocks_df = pd.DataFrame(stock_sentiments, columns = ['symbol', 'date','text', 'compound', 'positive', 'negative', 'neutral']) 

In [79]:
# Top 10 in the dataframe

nlp_stocks_df.head(10)

,symbol,date,text,compound,positive,negative,neutral
0,ABMD,2021-04-08T11:06:16Z,"DANVERS, Mass.--(BUSINESS WIRE)--Abiomed, Inc....",0.0000,0.000,0.000,1.000
1,ABMD,2021-03-16T15:39:23Z,<ul><li>According to an update posted on the f...,-0.4767,0.000,0.094,0.906
2,ABMD,2021-04-08T14:45:20Z,The financial regulations require hedge funds ...,0.3612,0.072,0.000,0.928
3,ABMD,2021-04-06T17:42:16Z,The Insider Monkey team has completed processi...,0.0000,0.000,0.000,1.000
4,ABMD,2021-03-29T16:18:22Z,The 800+ hedge funds and famous money managers...,0.0000,0.000,0.000,1.000
5,ABMD,2021-03-26T14:00:00Z,Abstract\r\nBackground\r\nRheumatoid arthritis...,0.0000,0.000,0.000,1.000
6,ABMD,2021-03-21T11:52:28Z,Photo by Hoptocopter/E+ via Getty Images\r\nIn...,0.4023,0.085,0.000,0.915
7,FLIR,2021-04-01T12:00:00Z,Its cheaper than its predecessor with even mor...,0.0000,0.000,0.000,1.000
8,FLIR,2021-04-05T23:00:10Z,"With how expensive thermal cameras are, why no...",0.6249,0.100,0.000,0.900
9,FLIR,2021-04-01T17:09:37Z,Have the need or in the mood for a thermal-see...,0.0000,0.000,0.000,1.000


In [80]:
# Last 10 in the dataframe

nlp_stocks_df.tail(10)

,symbol,date,text,compound,positive,negative,neutral
1945,JCI,2021-03-30T17:37:10Z,"With Treasury yields back on the rise, the sto...",-0.1027,0.051,0.063,0.886
1946,JCI,2021-03-15T21:03:07Z,Honeywell International Inc. (NYSE:HON) JP Mor...,0.3680,0.086,0.000,0.914
1947,JCI,2021-04-07T13:23:00Z,Advertisement\r\nJapanese Prime Minister Suga ...,0.0000,0.000,0.000,1.000
1948,JCI,2021-03-29T08:52:53Z,This article is part of a series that provides...,0.0000,0.000,0.000,1.000
1949,JCI,2021-04-06T14:00:00Z,Abstract\r\nBackground and aims\r\nNeutrophil-...,0.0258,0.084,0.080,0.836
1950,JCI,2021-03-16T14:00:00Z,Abstract\r\nBackground\r\nResearch surrounding...,0.0000,0.000,0.000,1.000
1951,JCI,2021-03-30T14:00:00Z,Abstract\r\nLignosus rhinocerotis Cooke. (L. r...,0.0000,0.000,0.000,1.000
1952,JCI,2021-04-01T14:00:00Z,Abstract\r\nDue to the durability and persiste...,0.0000,0.000,0.000,1.000
1953,JCI,2021-04-01T14:00:00Z,Abstract\r\nThe particle size distribution (PS...,0.0000,0.000,0.000,1.000
1954,JCI,2021-03-30T15:07:00Z,"If youre considering retirement abroad, you ne...",0.0000,0.000,0.000,1.000


In [81]:
# Group by symbol in nlp_stocks dataframe with mean of positive sentiments and convert into nlp_stocks_pos dataframe

nlp_stocks_pos = nlp_stocks_df.groupby('symbol')['positive'].mean()

nlp_stocks_pos = pd.DataFrame(nlp_stocks_pos)
nlp_stocks_pos

,positive
symbol,
ABMD,0.022429
ABT,0.083340
ADP,0.088050
ADSK,0.098070
AXP,0.066465
BSX,0.048195
CF,0.087180
CME,0.148870
CPB,0.054780


In [82]:
# # Top 5 tickers by avarage positive sentiment in last 30 days    

top_5_stocks = nlp_stocks_pos['positive'].nlargest(n=5)
top_5_stocks

symbol
ULTA    0.157400
CME     0.148870
TSCO    0.119483
FBHS    0.117750
ESS     0.107480
Name: positive, dtype: float64

In [83]:
# A list of top 5 tickers by avarage positive sentiment in last 30 days

top_5_stocks_list = top_5_stocks.index.tolist()
top_5_stocks_list

['ULTA', 'CME', 'TSCO', 'FBHS', 'ESS']

In [84]:
# Create a function to get key statistics of top 5 tickers by avarage positive sentiment in last 30 days

def getStockStats(symbol):
 
    url = r'https://finance.yahoo.com/quote/{0}/key-statistics?p={0}'.format(symbol)
    
    dfList = pd.read_html(requests.get(url).text)
        
    top_5_df = dfList[0] 
    for stats in dfList[1:]:
        top_5_df = top_5_df.append(stats)

    return top_5_df.set_index(0).T 

In [85]:
# Get key statistics of top 5 tickers by avarage positive sentiment in last 30 days

top_5_stocks_list = ['CME', 'ULTA', 'FBHS', 'TSCO', 'ADSK']

for i, ticker in enumerate(top_5_stocks_list):
    if i == 0:
        dfStats = getStockStats(ticker)
        dfStats["Symbol"] = ticker
        top_5_stats = dfStats
    else:
        dfStats = getStockStats(ticker)
        dfStats["Symbol"] = ticker
        top_5_stats = top_5_stats.append(dfStats)



top_5_stats = top_5_stats.set_index("Symbol")

In [86]:
top_5_stats

,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 7,Beta (5Y Monthly),...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
Symbol,,,,,,,,,,,,,,,,,,,,,
CME,73.3B,75.83B,34.78,28.00,6.22,15.05,2.78,15.57,24.17,0.47,...,5.87,-9.70%,1.73B,4.83,4.07B,15.45,1.01,73.50,2.72B,1.99B
ULTA,18.31B,18.96B,104.77,27.13,0.79,2.98,9.17,3.08,27.07,1.71,...,3.11,-23.00%,1.05B,18.61,1.9B,94.86,1.87,35.54,810.36M,699.05M
FBHS,14.05B,16.29B,25.76,18.42,1.98,2.31,5.07,2.67,16.02,1.65,...,3.94,57.20%,419.1M,3.02,2.75B,99.12,1.64,20.02,825.7M,566.95M
TSCO,20.4B,22.87B,27.52,24.31,2.27,1.92,10.61,2.15,16.28,1.03,...,6.38,-5.70%,1.34B,11.54,3.55B,184.32,1.87,16.55,1.39B,974M
ADSK,64.69B,65.59B,54.15,42.45,1.49,17.07,67.00,17.31,88.40,1.34,...,5.44,591.40%,1.86B,8.46,2.1B,217.98,0.83,4.40,1.44B,1.24B
